# Building an auto insurance chatbot using Azure OpenAI ChatGPT

### Problem description
    Build an Auto insurance chatbot to help users by answering their questions about auto insurance plans and providing personalized recommendations based on the user account information.

    Within the chatbot workflow, this model is placed after the topic classifier (which distinguishes the topic of a user input), and is only responsible for handling user utterances that are relevant to auto insurance questions.
### Background
    The chatbot is for an insurance company called Contoso, Ltd.
    The chatbot has access to relevant FAQ knowledge base on auto insurance, as well as customers information.
    
    An example on customer information shown below:
        Insurer's Name: Dave Huang
        Membership Start Date: March 2019
        Auto Insurance: Yes
        Auto Insurance Deductible: $500 per incident
        Auto Insurance Claims history: auto accident in Jan 2020; auto accident in Apr 2022 
        Safe Driver Rating: 22 of 100
        Home Insurance: No
        Flood Insurance: No
        Home Address: 100 Main Street, Seattle, WA
        Flood Risk Factor: 8 of 10
        Bundling Discount: No
        Safe Driver Discount: No
            
### Approach
    The model is built using Azure OpenAI ChatGPT-turbo base model, relevant information (insurance knowledge and user account information) is provided in the system prompt.

### Result
    The chatbot is working as expected, it can provide personalized answers based on knowledge base and user account information. A few manual evaluation examples on two user accounts are included.

**note**: This notebook is built using the March 2023 version of ChatGPT (gpt-35-turbo). As ChatGPT version is updated roughly once a month, please re-evaluate your prompt engineering if you are using a version that is more recent than March 2023.

# set up 

In [91]:
import os
from openai import AzureOpenAI
import json

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

chatgpt_model_name = config_details['CHATGPT_MODEL']
openai.api_type = "azure"
openai.api_key = config_details['OPENAI_API_KEY']
openai.api_base = config_details['OPENAI_API_BASE']
openai.api_version = config_details['OPENAI_API_VERSION']


# prompt engineering + API calls

All chat history relevant to the topic of auto insurance is combined for ChatGPT to "remember" and be aware of the dialogue state.

In [92]:
def engineer_user_prompt(user_input):
    messages = []
    messages.append({"role": "user", "content": user_input})

    return messages

In [97]:
def engineer_prompt(user_input, lst_user_bot_input):
    '''
    prompt engineering so the bot has context of previous conversations
    
    continue appending previous conversations so the lateste prompt include all previous chat history
    '''
    messages = []
    for item in lst_user_bot_input:
        #messages.append({"role": "user", "content": item})
        messages.append(item)
        print(item)
    
    # Add the current user input
    messages.append({"role": "user", "content": user_input})

    
    return messages


In [98]:
def call_chatgpt(prompt_input, deployment_name=chatgpt_model_name):
    '''
    call ChatGPT API, and set parameters, return useful output from API call
    '''
    client = AzureOpenAI(
      azure_endpoint = openai.api_base, 
      api_key=openai.api_key,  
      api_version=openai.api_version
    ) 

    # Extract messages from the prompt_input dictionary
    messages = [msg['content'] for msg in prompt_input['messages']]
    
    response = client.chat.completions.create(
      model="WNChatbotDevelopment", # model = "deployment_name"
      messages=messages,
      temperature=0,
      max_tokens=500,
      top_p=0.95,
      frequency_penalty=0.5,
      presence_penalty=0.0,
      stop=None 
    )
    
    return response["choices"][0]['text']



In [99]:
user_dave = """
        Insurer's Name: Dave Huang
        Membership Start Date: March 2019
        Auto Insurance: Yes
        Auto Insurance Deductible: $500 per incident
        Auto Insurance Claims history: auto accident in Jan 2020; auto accident in Apr 2022 
        Safe Driver Rating: 22 of 100
        Home Insurance: No
        Flood Insurance: No
        Home Address: 100 Main Street, Seattle, WA
        Flood Risk Factor: 8 of 10
        Bundling Discount: No
        Safe Driver Discount: No
        Driving Habit Tracking Device Discount: No
        """
user_emma = """
        Insurer's Name: Emma Miller
        Membership Start Date: December 2017
        Auto Insurance: Yes
        Auto Insurance Deductible: $500 per incident
        Auto Insurance Claims history: None
        Safe Driver Rating: 90 of 100
        Home Insurance: Yes
        Home Flood Insurance: No
        Home Address: 44, 170th Ave, Redmond, WA
        Flood Risk Factor: 1 of 10
        Bundling Discount: Yes
        Safe Driver Discount: Yes
        Driving Habit Tracking Device Discount: No
        """
lst_users = [user_dave, user_emma]

In [100]:
def engineer_system_prompt(user):
    '''
    build system prompt, add relevant information into it
    '''
    
    prompt_system = f"""
    As an auto insurance agent at Contoso, Assistant's role is to help users by answering their questions about auto insurance plans and providing personalized recommendations based on the User Account Information (Content A). 
    Assistant should never refer the user to any other insurance companies, repeat previously stated sentences or information, or provide quotes. 
    Assistant is a passive advisor; Assistant doesn't ask user questions.

    Content A. Ways to Lower Insurance Premium:  
        1. Bundling Discount. Bundling discount is applied when you have more than one insurance plan with Contoso. Prioritize recommend bundling if the user's Bundling Discount is No.
        2. Increase Deductible. The higher your deductible, the lower your premium will be. $500 is the average.
        3. Safe Driver Discount. Maintain a high safe driver score, higher than 70 of 100 is eligible for a discount. The higher rating you have, the lower your premium will be. 
        4. Driving Habit Tracking Device Discount. Install a driving habit tracking device in your vehicle to get a discount.
    
    If a user wants to get a quote or asks about cost of premium, Assistant should reply and only reply with "Let me connect you with a quote specialist at Contoso who can then help you further. Please hold while I connect you." and say nothing else.
    Assistant has access to all Account Information for the user from Content A. When responding to user questions, Assistant should combine relevant information from all 4 points listed in Content B (Ways to Lower Insurance Premium) and personalized recommendations from Content A. 
    Assistant should always go through all 4 points one by one in Ways to Lower Insurance Premium and determine if it applies to the user based on User Account Information (Content A). If the point does not apply, explain what the user needs to do in order to qualify. Prioritize recommend bundling if the user currently don't have Bundling Discount.
    """
    messages = []
    messages.append({"role": "system", "content": prompt_system})

    return messages

In [101]:
def engineer_lst_questions(user):
    '''
    put together system prompt and pre-programmed user questions input in sequence
    '''
    lst_user_bot_input = [engineer_system_prompt(user)]

    user_input_1 = """What are some ways for me to lower my insurance premium?"""
    user_input_2 = """Can you tell me more about bundling?"""
    user_input_3 = """Thank you for the detailed explanation. I would like to get a quote with bundling auto and home insurance""" 

    lst_user_inputs = [user_input_1, user_input_2, user_input_3]

    return lst_user_bot_input, lst_user_inputs

In [107]:
def display_conversation(user, tag_display_code=False):
    '''
    call ChatGPT API and display User questions and ChatGPT replies in conversational style
    making it easier to evaluate the results
    '''
    lst_user_bot_input, lst_user_inputs = engineer_lst_questions(user)
    for i, user_input_i in enumerate(lst_user_inputs):
        prompt_input_i = engineer_prompt(user_input_i, lst_user_bot_input)
        #print(prompt_input_i)
        #response_i = call_chatgpt(prompt_input_i)
        #lst_user_bot_input.append(response_i)
        # display output along each round of conversation
        #print(f"{user_input_i}\n---------------\n{response_i}\n---------------")
        
    #if tag_display_code: #display all prompt (code style)
        #print(f"\n==============================\nCode style Prompt and Responses\n==============================\n{prompt_input_i}{response_i}")

    print(lst_user_bot_input)


In [108]:
for user in lst_users:
    print(f"\n==============================\n{user}\n==============================\n")
    display_conversation(user, tag_display_code=True)



        Insurer's Name: Dave Huang
        Membership Start Date: March 2019
        Auto Insurance: Yes
        Auto Insurance Deductible: $500 per incident
        Auto Insurance Claims history: auto accident in Jan 2020; auto accident in Apr 2022 
        Safe Driver Rating: 22 of 100
        Home Insurance: No
        Flood Insurance: No
        Home Address: 100 Main Street, Seattle, WA
        Flood Risk Factor: 8 of 10
        Bundling Discount: No
        Safe Driver Discount: No
        Driving Habit Tracking Device Discount: No
        

[{'role': 'system', 'content': '\n    As an auto insurance agent at Contoso, Assistant\'s role is to help users by answering their questions about auto insurance plans and providing personalized recommendations based on the User Account Information (Content A). \n    Assistant should never refer the user to any other insurance companies, repeat previously stated sentences or information, or provide quotes. \n    Assistant is a passive advis